In [ ]:
# self note -> yeah no go/ No voice cloning or model creation idts

In [5]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [6]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "zhongli_voice_samples/")
)

In [7]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    eval_split_size=0.2,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

In [8]:
# import wave

# # Specify the path to your WAV file
file_path = "tts_train_dir/zhongli_voice_samples/wavs/VO_Zhongli_Chat_-_Journeys.wav"

# # Open the WAV file
# with wave.open(file_path, "rb") as wav_file:
#     # Get the sample rate
#     sample_rate = wav_file.getframerate()
    
#     print("Sample rate:", sample_rate)

import soundfile as sf

# Specify the path to your WAV file
# file_path = "path/to/your/file.wav"

# Read the WAV file and get the sample rate
data, sample_rate = sf.read(file_path, always_2d=True)

print("Sample rate:", sample_rate)


Sample rate: 48000


In [9]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
ap.sample_rate = 48000


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [10]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [11]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 5 files in C:\Users\advai\Super Secret Nuero Folder\tts_train_dir\zhongli_voice_samples


In [12]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [13]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Num. of CPUs: 16
 | > Num. of Torch Threads: 8
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=tts_train_dir\run-June-13-2023_04+18PM-0000000

 > Model has 28610257 parameters


In [14]:
trainer.fit()


 > EPOCH: 0/1
 --> tts_train_dir\run-June-13-2023_04+18PM-0000000

 > TRAINING (2023-06-13 16:18:17) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 4
 | > Preprocessing samples
 | > Max text length: 262
 | > Min text length: 45
 | > Avg text length: 150.0
 | 
 | > Max audio length: 165549.5
 | > Min audio length: 27135.0
 | > Avg audio length: 91279.5
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:/Users/advai/Super Secret Nuero Folder/tts_train_dir/run-June-13-2023_04+18PM-0000000\\events.out.tfevents.1686653296.advait-tuf'

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
ckpts

[]

In [ ]:
configs

[]

In [ ]:
 !tts --text "Text for TTS" \
      --model_path ckpt[0] \
      --config_path configs[0] \
      --out_path out.wav

Traceback (most recent call last):
  File "C:\Users\advai\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\advai\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\advai\anaconda3\Scripts\tts.exe\__main__.py", line 7, in <module>
  File "C:\Users\advai\anaconda3\lib\site-packages\TTS\bin\synthesize.py", line 371, in main
    synthesizer = Synthesizer(
  File "C:\Users\advai\anaconda3\lib\site-packages\TTS\utils\synthesizer.py", line 86, in __init__
    self._load_tts(tts_checkpoint, tts_config_path, use_cuda)
  File "C:\Users\advai\anaconda3\lib\site-packages\TTS\utils\synthesizer.py", line 144, in _load_tts
    self.tts_config = load_config(tts_config_path)
  File "C:\Users\advai\anaconda3\lib\site-packages\TTS\config\__init__.py", line 91, in load_config
    raise TypeError(f" [!] Unknown config file type {ext}")
TypeError:  [!] Unknown config file type 


In [ ]:
import IPython
# IPython.display.Audio(file_path)
IPython.display.Audio("out.wav")